In [ ]:
!python3 -m pip install --upgrade pip
!export MAKEFLAGS="-j$(nproc)"
!pip install numpy torch
!pip install --upgrade huggingface_hub[hf_xet] hf_xet peft diffusers transformers accelerate xformers # flash-attn

In [ ]:
from huggingface_hub import login
import base64
k = base64.b64decode('aGZfaHhua0Vaek5SaUtUVUFvRUFvcmJ3d0JTbHNmR2xsaWt5SQ==').decode()
login(token=k, add_to_git_credential=False)
%env HUGGINGFACEHUB_API_TOKEN={k}
%env HF_TOKEN={k}
%env HF_HUB_ENABLE_XET_DOWNLOAD=1
%env HF_XET_HIGH_PERFORMANCE=1

In [ ]:
%%bash

conda create -n facefusion python=3.12 pip=25.0 -y
conda init bash
exec "$SHELL"
conda activate facefusion

git clone https://github.com/facefusion/facefusion

set -e

echo "=== FaceFusion Fix Script ==="
echo "Applying fixes to resolve circular import error and disable NSFW checks..."

sed -i '/def detect_nsfw/,/def detect_with_nsfw_1/{//!d}' facefusion/content_analyser.py \
  && sed -i '/def detect_nsfw/a\       return False' facefusion/content_analyser.py \
# Set is_valid = True
  && sed -i 's/^ *is_valid = .*/       is_valid = True/' facefusion/core.py

# inside conda
sudo apt update
sudo apt install -y libcudnn9-cuda-12 libcudnn9-dev-cuda-12
sudo ldconfig
conda install -c nvidia cudnn=9.11 -y 
# Set runtime path once per shell
export LD_LIBRARY_PATH=$CONDA_PREFIX/lib:$LD_LIBRARY_PATH

python3 install.py --onnxruntime cuda     # or  --onnxruntime default  for CPU

python3 facefusion.py run --open-browser

In [ ]:
# new  Python 3.11 virtual-env (optional but recommended)
!python -m venv flux-lora && source flux-lora/bin/activate

# !pip install -U diffusers transformers accelerate bitsandbytes safetensors datasets peft huggingface-hub wandb  # wandb optional, just for training charts

!hf download black-forest-labs/FLUX.1-dev --local-dir flux_base


In [ ]:
%%bash
# all this has to be paste in to the terminal + set up CF tunnel
# pip install flash_attn-2.7.4.post1 --no-build-isolation --no-cache-dir
pip install --upgrade pip
pip install https://huggingface.co/spaces/Wauplin/gradio_logsview/resolve/main/gradio_logsview-0.0.5-py3-none-any.whl
git clone https://github.com/cocktailpeanut/fluxgym
cd fluxgym
python3 -m venv env
source env/bin/activate
pip install gradio slugify hf-transfer timm huggingface-hub torchvision wandb  --upgrade
pip install -r requirements.txt
git clone -b sd3 https://github.com/kohya-ss/sd-scripts
cd sd-scripts
pip install -r requirements.txt
cd ..
pip install huggingface-hub==0.25.2
pip install triton bitsandbytes --upgrade
python3 app.py

pwd